In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import seaborn as sns

In [50]:
# Cargamos el dataset en una estructura DataFrame de Pandas
data = pd.read_excel(r"C:\Users\Santi\OneDrive\Escritorio\Proyectos DTS\Datathon\E-Commerce_train.xlsx")
data.drop(columns=['ID'], inplace=True)
data.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,D,Flight,4,2,177,3,low,F,44,1233,1
1,F,Flight,4,5,216,2,low,M,59,3088,1
2,A,Flight,2,2,183,4,low,M,48,3374,1
3,B,Flight,3,3,176,4,medium,M,10,1177,1
4,C,Flight,2,2,184,3,medium,F,46,2484,1


In [51]:
#SEPARAMOS CATEGORICOS DE NUMERICOS

categoricos = [cat for cat in data.columns if data[cat].dtype == 'object' and data[cat].nunique() < 10]
numericos = [num for num in data.columns if data[num].dtype != 'object']

Separando en caracteristicas y variables objetivos

In [52]:
#SACAMOS EL GENERO Y DEFINIMOS LO QUE SE QUIERE PREDECIR

X = data.loc[:,[dx for dx in data.columns if dx not in ['Reached.on.Time_Y.N', 'Gender']]]
y = data['Reached.on.Time_Y.N']

In [53]:
X.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Discount_offered,Weight_in_gms
0,D,Flight,4,2,177,3,low,44,1233
1,F,Flight,4,5,216,2,low,59,3088
2,A,Flight,2,2,183,4,low,48,3374
3,B,Flight,3,3,176,4,medium,10,1177
4,C,Flight,2,2,184,3,medium,46,2484


Separando valores en train test

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
#SEPARAMOS LOS VALORES DE PRUEBA Y ENTRENAMIENTO

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

Pipelines

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate


OrdinalEncoder  

In [57]:
encoder = OrdinalEncoder()
standard = StandardScaler() 

In [58]:
#TRANSFORMACION DE COLUMNAS

encoder_standard = ColumnTransformer(transformers=[
    ('encoder', encoder, categoricos[:3]),
    ('standar', standard, [num for num in numericos if num not in ['Reached.on.Time_Y.N']])
])

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [60]:
parans_forest = {
    'n_estimators': range(10,30),
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(10,30)
}

In [61]:
modeloForest = RandomForestClassifier(random_state=1)
gridForest = GridSearchCV(modeloForest, parans_forest, scoring='recall')

SVM

Ordinal

In [62]:
from sklearn.svm import SVC

In [63]:
parametros_svm = {
    'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
    'gamma': ('scala','auto')
}

In [64]:
modelSVC = SVC(random_state=1)
gridSVC = GridSearchCV(modelSVC, parametros_svm, scoring='recall')

In [65]:
pipeline_svc2 = Pipeline(steps=[ 
    ('previo',encoder_standard),
    ('gridSVC', gridSVC)
])

In [66]:
#ENTRENEMOS AL MODELO

pipeline_svc2.fit(train_x,train_y)

c:\Users\Santi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Santi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Santi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py", line 226, in fit
    raise ValueError(
ValueError: When 'gamma' is a string, it should be either 'scale' or 'auto'. Got ''scala'' instead.

  warn

Pipeline(steps=[('previo',
                 ColumnTransformer(transformers=[('encoder', OrdinalEncoder(),
                                                  ['Warehouse_block',
                                                   'Mode_of_Shipment',
                                                   'Product_importance']),
                                                 ('standar', StandardScaler(),
                                                  ['Customer_care_calls',
                                                   'Customer_rating',
                                                   'Cost_of_the_Product',
                                                   'Prior_purchases',
                                                   'Discount_offered',
                                                   'Weight_in_gms'])])),
                ('gridSVC',
                 GridSearchCV(estimator=SVC(random_state=1),
                              param_grid={'gamma': ('scala', 'auto'),
                                          'kernel': ('linear', 'poly', 'rbf',
                                                     'sigmoid')},
                              scoring='recall'))])

In [67]:
from sklearn.metrics import confusion_matrix,recall_score
pred_svc = pipeline_svc2.predict(test_x)
recall_score(test_y,pred_svc)

0.7291049199762892

In [68]:
#CARGAMOS EL NUEVO DATASET EN UNA ESTRUCTURA DATAFRAME DE PANDAS

ecomerce = pd.read_excel(r'C:\Users\Santi\OneDrive\Escritorio\Proyectos DTS\Datathon\E-Commerce_test.xlsx')

In [69]:
ecomerce.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9000,F,Ship,5,1,189,2,medium,M,10,5248
1,9001,D,Ship,5,3,150,2,high,F,9,4446
2,9002,F,Ship,3,3,200,3,low,M,3,5739
3,9003,A,Ship,5,1,133,2,medium,M,1,4643
4,9004,B,Ship,4,2,260,3,low,M,4,5504


In [70]:
#ACOMODAMOS LOS FEATURES

categoricos_e = [cat for cat in ecomerce.columns if ecomerce[cat].dtype == 'object' and ecomerce[cat].nunique() < 10]
numericos_e = [num for num in ecomerce.columns if ecomerce[num].dtype != 'object']

In [71]:
categoricos_e

['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']

In [72]:
numericos_e

['ID',
 'Customer_care_calls',
 'Customer_rating',
 'Cost_of_the_Product',
 'Prior_purchases',
 'Discount_offered',
 'Weight_in_gms']

In [73]:
ecomerce_test = ecomerce[numericos_e+categoricos_e]

In [74]:
ecomerce_test.head()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
0,9000,5,1,189,2,10,5248,F,Ship,medium,M
1,9001,5,3,150,2,9,4446,D,Ship,high,F
2,9002,3,3,200,3,3,5739,F,Ship,low,M
3,9003,5,1,133,2,1,4643,A,Ship,medium,M
4,9004,4,2,260,3,4,5504,B,Ship,low,M


In [75]:
#PREDECIMOS

pred_final = pipeline_svc2.predict(ecomerce_test)

In [76]:
df = pd.DataFrame({'pred':pred_final})

In [77]:
#EXPORTAMOS A CSV

df.to_csv('./santigll.csv',index=False)